In [27]:
# load data from a csv file at location: ../sample-data/data.csv

import pandas as pd

# Specify the file path
file_path = '../sample-data/data.csv'

# Use pandas to read the CSV file
data = pd.read_csv(file_path)

# Print the first few rows of the DataFrame
print(data.head())


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [ ]:
# write a query to create a table in the database called some_data and load the data from the csv file into the table
import sqlite3
import pandas as pd

# connect to the database
conn = sqlite3.connect("somedata.db")
cursor = conn.cursor()

# create a table named some_data with the same columns as the csv file
# Get the column names and data types from the DataFrame
column_names = data.columns.tolist()
data_types = data.dtypes.tolist()

# Create the CREATE TABLE statement
create_table_query = "CREATE TABLE some_data ("

# Iterate over the column names and data types to create the table schema
for column_name, data_type in zip(column_names, data_types):
    create_table_query += f"{column_name} {data_type},"

# Remove the trailing comma
create_table_query = create_table_query.rstrip(",")

# Add the closing parenthesis
create_table_query += ")"

# Execute the CREATE TABLE query
cursor.execute(create_table_query)


In [37]:
# Execute the PRAGMA statement to get the column information
cursor.execute("PRAGMA table_info(some_data)")

# Fetch all the rows returned by the query
rows = cursor.fetchall()

# Extract the column names from the rows
column_headers = [row[1] for row in rows]

# Print the column headers
print(column_headers)


['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']


In [38]:
# Read the rows from the CSV file
data = pd.read_csv(file_path)

# Write the rows to the SQLite database
data.to_sql('some_data', conn, if_exists='replace', index=False)


8000

In [39]:
# Count the number of rows in the CSV file
csv_row_count = len(data)

# Execute a query to count the number of rows in the SQL table
cursor.execute("SELECT COUNT(*) FROM some_data")
sql_row_count = cursor.fetchone()[0]

# Validate the congruency of the row counts
if csv_row_count == sql_row_count:
    print("The number of rows in the CSV file and SQL table are congruent.")
else:
    print("The number of rows in the CSV file and SQL table are not congruent.")


The number of rows in the CSV file and SQL table are congruent.


In [40]:
csv_row_count = len(data)
print("The number of rows in the CSV file is:", csv_row_count)


The number of rows in the CSV file is: 8000


In [41]:
# Execute a query to count the number of rows in the SQL table
cursor.execute("SELECT COUNT(*) FROM some_data")

# Fetch the result
row_count = cursor.fetchone()[0]

# Print the number of rows
print("The number of rows in the SQLite database is:", row_count)


The number of rows in the SQLite database is: 8000


In [42]:
%pip install pyodbc
%pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [43]:
import pyodbc
import os

# Define the connection string
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')
driver = '{ODBC Driver 17 for SQL Server}'  # Use the appropriate driver for your system

# Create the connection string
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"


In [44]:
# copy the data from the sqlite database to the sql server database
# Establish a connection to the SQLite database
sqlite_conn = sqlite3.connect("somedata.db")
sqlite_cursor = sqlite_conn.cursor()

# Establish a connection to the SQL Server database
sql_server_conn = pyodbc.connect(conn_str)
sql_server_cursor = sql_server_conn.cursor()

# Execute a query to select all the rows from the SQLite table
sqlite_cursor.execute("SELECT * FROM some_data")

# Fetch all the rows returned by the query
rows = sqlite_cursor.fetchall()

# Execute the INSERT query to insert the rows into the SQL Server table
sql_server_cursor.executemany("INSERT INTO some_data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", rows)

# Commit the transaction
sql_server_conn.commit()

# Close the connections
sqlite_conn.close()
sql_server_conn.close()


OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')

In [ ]:
import requests

# Make a GET request to the API
response = requests.get('https://api.ipify.org?format=json')

# Extract the IP address from the response
ip_address = response.json()['ip']

# Print the IP address
print("Your public IP address is:", ip_address)


Your public IP address is: 40.87.55.113
